In [10]:
import yfinance as yf
import pandas as pd
import os
import pandas_ta as ta
import numpy as np
from sklearn.preprocessing import StandardScaler

The strategy here is identical to the one in final.py .

In [4]:
ticker="^NSEI"

In [9]:
def data(ticker):                           
  df = yf.Ticker(ticker)
  df = df.history(period="max")
  df.index = pd.to_datetime(df.index)
  del df["Dividends"]
  del df["Stock Splits"]
  df = df.loc["2014-01-01":].copy()
  df["Target"] = (df["Close"] > df["Open"]).astype(int).shift(-1)   ##Binary target...tomorrow increase or decrease direction as an output to be decided by todays statistics
  # frequency = df['Target'].value_counts()
  # print(frequency)
  df.dropna(inplace=True)
  return df

In [6]:
# Calculating technical indicators using pandas_ta
def add_features(df):
    df['EMA_2'] = ta.ema(df['Close'], length=2)
    df['EMA_5'] = ta.ema(df['Close'], length=5)
    df['EMA_20'] = ta.ema(df['Close'], length=20)
    df['EMA_100'] = ta.ema(df['Close'], length=100)

    df['RSI_14'] = ta.rsi(df['Close'], length=14)
    macd = ta.macd(df['Close'])
    df['MACD_signal'] = macd['MACDs_12_26_9']
    df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'], length=14)
    df['OBV'] = ta.obv(df['Close'], df['Volume'])

    # Lagged features (kind of last few rows to predict the next row)
    for lag in range(1, 10):  
    # lag-=1
        df[f'Close_t-{lag}'] = df['Close'].shift(lag)
        df[f'Open_t-{lag}'] = df['Open'].shift(lag)
        df[f'High_t-{lag}'] = df['High'].shift(lag)
        df[f'Low_t-{lag}'] = df['Low'].shift(lag)
        df[f'Volume_t-{lag}'] = df['Volume'].shift(lag)

    df = df.dropna()
    return df

In [7]:
df=data(ticker)
df_added=add_features(df)

NameError: name 'yf' is not defined

In [ ]:
# Define lagged features and technical indicators
lagged_features = [f'Close_t-{i}' for i in range(1, 10)] + [f'Open_t-{i}' for i in range(1, 10)] + [f'Volume_t-{i}' for i in range(1, 10)]
featuresq = ['EMA_2', 'EMA_5','EMA_20','EMA_100', 'RSI_14', 'MACD_signal', 'ATR', 'OBV', 'Close', 'Volume']

# Combine all features
all_features = featuresq + lagged_features
# Feature normalization
scaler = StandardScaler()
df[all_features] = scaler.fit_transform(df[all_features])

# Preparing the features and target
X = df_added[all_features]
y = df_added['Target']  

The strategy is simple. The model will be trained for the last 350 days of each day, and predict the direction of price for that day. It will buy/sell accordingly at the opening price of the day and clear the position at the end of each day. It will always use atmost a certain percentage of cash to place the orders. An order log will be created and each order will be displayed. If cash is not enough to place an order, or clear a position, emergency account balance will be used accordingly and the trading process may get terminated.

In [ ]:
import os

portfolio = 10000  # Starting with Rs. 10,000
def simulate_trading(df, model, all_features, transaction_cost=0.01):
    cash = portfolio
    position = 0
    buy_price = 0
    predictions = []
    
    log_file = 'trading_log.txt'
    with open(log_file, 'w') as file:
        file.write('Date,Action,Price,Amount,Portfolio Value\n')
        
        for i in range(355, len(df)):  # Start from day 355 to ensure sufficient data
            features = df.iloc[i][all_features].values.reshape(1, -1)
            prediction = model.predict(features)[0]
            close_price = df['Close'].iloc[i]
            predictions.append(prediction)
            date = df.index[i]
            
            if prediction == 1:  # Buy signal
                if position == 0:  # No position open
                    # Buy
                    amount_to_buy = cash / close_price
                    cash -= amount_to_buy * close_price * (1 + transaction_cost)
                    position = amount_to_buy
                    buy_price = close_price
                    action = 'BUY'
                else:
                    action = 'HOLD'
            else:  # Sell signal
                if position > 0:  # Position open
                    # Sell
                    cash += position * close_price * (1 - transaction_cost)
                    position = 0
                    action = 'SELL'
                else:
                    action = 'HOLD'
            
            # Calculate current portfolio value
            portfolio_value = cash + position * close_price
            
            # Print order details and portfolio value
            print(f"Date: {date}, Action: {action}, Price: {close_price:.2f}, Amount: {position:.2f}, Portfolio Value: ${portfolio_value:.2f}")
            
            # Log the order details
            with open(log_file, 'a') as file:
                file.write(f'{date},{action},{close_price:.2f},{position:.2f},{portfolio_value:.2f}\n')
    
    # Closing any remaining position
    if position > 0:
        cash += position * df['Close'].iloc[-1] * (1 - transaction_cost)
    
    profit = cash - portfolio
    df['Prediction'] = [np.nan] * 350 + predictions
    return df, profit

# Simulate trading
df_with_predictions, profit = simulate_trading(df, model3, all_features)
print(f"Total profit over the year: ${profit:.2f}")
